In [19]:
import pandas as pd
import numpy as np

df_a=pd.read_excel('~/Desktop/销售客流pandas/1-12月份数据.xlsx',sheet_name='A')
m = df_a.groupby(['公司名称']).count().reset_index()

df_b=pd.read_excel('~/Desktop/销售客流pandas/1-12月份数据.xlsx',sheet_name='B')
n = df_b.groupby(['公司名称']).count().reset_index()

for i in m['公司名称']:
    df_t = df_a.loc[(df_a['公司名称']==i)]
    df_t.to_excel('~/Downloads/待调剂_无预算_超预算/预算待调剂_'+i+'_全年无预算.xlsx',index=False,sheet_name='全年无预算')

for j in n['公司名称']:
    df_t = df_b.loc[(df_b['公司名称']==j)]
    df_t.to_excel('~/Downloads/待调剂_无预算_超预算/预算待调剂_'+j+'_全年超预算.xlsx',index=False,sheet_name='全年超预算')

In [ ]:

import pandas as pd
import numpy as np

df_kl=pd.read_excel('~/Downloads/kl-1.xlsx')
df_kl.insert(loc=0,column='序号',value = [1,5,6,2,4,3])
df_kl_t=df_kl.sort_values(by=['序号'],ascending=True)
page_2 = df_kl_t[['项目','日均客流','同期日均客流','日均客流同比','平日日均客流','周末日均客流','周末/平日比','同期周末/平日比']]
page_2.astype({'周末/平日比':'float','同期周末/平日比':'float'})
page_2['周末占比同比'] = df_kl_t['周末/平日比'] - df_kl_t['同期周末/平日比']
page_2 = df_kl_t[['项目','日均客流','同期日均客流','日均客流同比','平日日均客流','周末日均客流','周末/平日比','周末占比同比']]

df_xs=pd.read_excel('~/Downloads/xs-1.xlsx')
df_all = df_xs.loc[(df_xs.项目分类 =='项目整体')]
df_all.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_a=df_all.sort_values(by=['序号'],ascending=True)
page_3 = df_xs_a[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]


df_major = df_xs.loc[(df_xs.项目分类 =='主次力店')]
df_major.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_m=df_major.sort_values(by=['序号'],ascending=True)
page_4 = df_xs_m[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]


df_normal = df_xs.loc[(df_xs.项目分类 =='一般店铺')]
df_normal.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
df_xs_n=df_normal.sort_values(by=['序号'],ascending=True)
page_5 = df_xs_n[['项目','日均销售额','同期日均销售额','日均销额售同比','平日日均销售额','周末日均销售额','周末/平日比','周末占比同比']]



page_1 = pd.merge(page_2,page_3,on='项目',how='left')[['项目','日均客流','日均客流同比','日均销售额','日均销额售同比']]


rjkl=page_2.日均客流.sum()/6
rjkl_tq = page_2.同期日均客流.sum()
rjkl_tb = (rjkl-rjkl_tq)/rjkl_tq

rjxs=page_3.日均销售额.sum()/6
rjxs_tq=page_3.同期日均销售额.sum()
rjxs_tb=(rjxs-rjxs_tq)/rjxs_tq

df_a_row1 = pd.DataFrame(np.insert(page_1.values,0,values=['均值',rjkl_tq,rjkl_tb,rjxs_tq,rjxs_tb],axis=0))

page_2=page_2.round(decimals=1)
page_3=page_3.round(decimals=0)
page_4=page_4.round(decimals=0)
page_5=page_5.round(decimals=0)
page_1=page_1.round(decimals=0)

with pd.ExcelWriter('~/Downloads/ppt.xlsx') as writer:
    df_a_row1.to_excel(writer,sheet_name='page_1')
    page_2.to_excel(writer,sheet_name='page_2')
    page_3.to_excel(writer,sheet_name='page_3')
    page_4.to_excel(writer,sheet_name='page_4')
    page_5.to_excel(writer,sheet_name='page_5')
    


### 半年报销售数据

In [ ]:
from datetime import date, timedelta


def days_between(d1,d2):
    days_between = (d2-d1).days
    weekends, leftover = divmod(days_between,7)
    if leftover:
        start_day = (d2-timedelta(leftover)).isoweekday()
        end_day = start_day+leftover
        if start_day<=6 and end_day>6:
            weekends +=.5
        if start_day<=7 and end_day>7:
            weekends +=.5

    days_a = (d2 - d1).days
    days_w = weekends
    days_n =days_a - days_w*2
    
    return (days_a,days_n,days_w*2)




def get_diff_xs(df_xs_all_ppk1,df_xs_all_ppk2,form,mall,dt11,dt12,dt21,dt22):
    """
    Parameters: 
        1、dataframe: df_xs_all_ppk1 历史数据,df_xs_all_ppk2  当前数据 ；格式= ['项目名称','铺位类型','品牌名称','营业额','营业额日期','业态']
        
            get_diff_xs(df_xs_dj_202105,df_xs_dj_202205,'合肥弘阳广场',0)
        2、form:0-所有业态；1-主次力店；2-一般商铺
    Return :
    项目名称	 平日销售日均	 周末销售日均	周末/平日比	周末占比同比
    合肥弘阳广场	788503.48	1123773.68  	0.58	-0.39
    """
    #根据业态，区分销售数据的统计范围
    if form==1:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态=='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态=='主次力店')]
    if form==2:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态!='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态!='主次力店')]

    # 
    dw = days_between(date(2022,1,1),date(2022,7,1))
    print(dw)
    c_zm1 = dw[2]
    c_pr1 = dw[1]
    
    #month1 周末平日销售结果
    df_xs_all_ppk1['周末'] = df_xs_all_ppk1['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    print((c_zm1,c_pr1))
    
    df_tmp1 = df_xs_all_ppk1.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    
    df_tmp1['日均营业额']=df_tmp1.apply(lambda x:x['营业额']/c_zm1 if x['周末']=='是' else x['营业额']/c_pr1,axis=1)

    a,b,c=df_tmp1.iat[0,2],df_tmp1.iat[1,2],df_tmp1.iat[1,1]/df_tmp1.iat[0,1]
    df_xs_res1 = pd.DataFrame(data={'项目名称':[mall],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df1 = df_xs_res1.reset_index()

    #month2 周末平日销售结果
    df_xs_all_ppk2['周末'] = df_xs_all_ppk2['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    c_zm2 = dw[2]
    c_pr2 = dw[1]

    df_tmp2 = df_xs_all_ppk2.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    df_tmp2['日均营业额']=df_tmp2.apply(lambda x:x['营业额']/c_zm2 if x['周末']=='是' else x['营业额']/c_pr2,axis=1)

    print(df_tmp2)
    a,b,c=df_tmp2.iat[0,2],df_tmp2.iat[1,2],df_tmp2.iat[1,1]/df_tmp2.iat[0,1]

    df_xs_res2 = pd.DataFrame(data={'项目名称':[mall],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df2 = df_xs_res2.reset_index()

    #输出同比结果
    df2['上期周末占比']=df1.iat[0,4]
    df2['周末占比同比']=df2.iat[0,4]-df1.iat[0,4]

    return df2.round(2)

In [ ]:

import pandas as pd
import numpy as np

df_xs_yt=pd.read_excel('~/Desktop/销售客流pandas/业态202201_06.xlsx',header=1)
df_xs_yt = df_xs_yt.loc[(df_xs_yt.项目 == '南京弘阳广场') | (df_xs_yt.项目 == '合肥弘阳广场') | (df_xs_yt.项目 == '常州中吴弘阳广场') | (df_xs_yt.项目 == '济南弘阳广场') | (df_xs_yt.项目 == '燕郊弘阳广场') | (df_xs_yt.项目 == '衡阳弘阳广场')]

# df_xs_yt = df_xs_yt[['项目整体','项目','一月','二月','三月','四月','五月','六月']]
# # df_xs_yt
# yetai = ['主次力店','餐饮','儿童','服装','精品','娱乐','服务']

# df_res = []
# for yt in yetai:
#     df = df_xs_yt.loc[(df_xs_yt.项目整体 == yt)]
#     df.insert(loc=0,column='序号',value =[1,5,6,2,4,3])
#     df=df.sort_values(by=['序号'],ascending=True)
#     d = df[['项目整体','项目','一月','二月','三月','四月','五月','六月']]
#     x = d.apply(pd.to_numeric,errors='ignore')
#     df_res.append(x)

# with pd.ExcelWriter('~/Desktop/销售客流pandas/ppt7.xlsx') as writer:
#     i = 0
#     for yt in yetai:
#         df_res[i].round(decimals=0).to_excel(writer,sheet_name= yt,index=False)
#         i += 1

# 南京弘阳广场2021 和 2022 销售数据
df1 = pd.read_excel('~/Desktop/销售客流pandas/正铺202101_06.xls')
df2 = pd.read_excel('~/Desktop/销售客流pandas/多经202101_06.xls')
df3 = pd.read_excel('~/Desktop/销售客流pandas/正铺202201_06.xls')
df4 = pd.read_excel('~/Desktop/销售客流pandas/多经202201_06.xls')

df2021 = pd.concat([df1,df2])

df2022 = pd.concat([df3,df4])


df_ppk = pd.read_excel('~/Desktop/销售客流pandas/品牌库.xlsx')
df2021 =df2021[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
df_ppk = df_ppk[['品牌名称','业态']]
df2021_ppk = pd.merge(df2021,df_ppk,on='品牌名称',how='left')
df2021_ppk['月份'] = df2021_ppk['营业额日期'].dt.month
df2021_ppk_res = df2021_ppk.groupby(by=['项目名称','月份']).agg({'营业额':'sum'}).reset_index()
print(df2021_ppk_res.T)

df_ppk = pd.read_excel('~/Desktop/销售客流pandas/品牌库.xlsx')
df2022 =df2022[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
df_ppk = df_ppk[['品牌名称','业态']]
df2022_ppk = pd.merge(df2022,df_ppk,on='品牌名称',how='left')

df2022_ppk['月份'] = df2022_ppk['营业额日期'].dt.month
df2022_ppk_res = df2022_ppk.groupby(by=['项目名称','月份']).agg({'营业额':'sum'}).reset_index()



# with pd.ExcelWriter('~/Desktop/销售客流pandas/ppt4.xlsx') as writer:
#     i = 0
#     for yt in yetai:
#         dtmp = df2022_ppk_res.loc[(df2022_ppk_res['业态'].str.strip() == yt)]
#         dtmp['营业额'] = dtmp['营业额']/10000
#         dtmp.round(0)
#         dtmp.T.to_excel(writer,sheet_name= yt,index=False)
#         i += 1

d = days_between(date(2022,1,1),date(2022,7,1))

# 2022 1-6月 日均销售
def get_xs_rijun(df2022_pk):
    df2022_ppk = df2022_pk.groupby(by=['项目名称','营业额日期']).agg({'营业额':'sum'}).reset_index()
    df2022_ppk['营业额'] = df2022_pk['营业额']/10000
    dff = df2022_pk.groupby(by=['项目名称']).agg({'营业额':'sum'}).reset_index()
    dff['总销售额']  = dff['营业额']
    dff['日均销售额']=dff['营业额']/d[0]
    dff['天数'] = d[0]
    return dff

# 2022 1-6月 各业态销售
def get_xs_by_yt(df2022_pk):
    
    ## 1、按项目、业态、月份分组汇总
    df2022_pk['月份'] = df2022_pk['营业额日期'].dt.month
    df2022_pk_yt = df2022_pk.groupby(by=['项目名称','业态','月份']).agg({'营业额':'sum'}).reset_index()
    df_nj = df2022_pk_yt.sort_values(by=['项目名称','业态','月份'])
    from functools import reduce

    ## 2、行转列、合并、整理
    yetai = ['主次力店','餐饮','儿童','服装','精品','娱乐','服务']
    df_r = pd.DataFrame()
    dflist = []
    for yt in yetai :
        dtmp = df_nj.loc[(df_nj.业态.str.strip() == yt)][['月份','营业额']].T.reset_index()
        dtmp.columns = ['项目','一月','二月','三月','四月','五月','六月']
        dr = dtmp[1:].copy()
        dr['项目'] = '南京弘阳广场'
        dr[['项目整体']] = yt
        dflist.append(dr)
    
    df_res = reduce(lambda left,right:pd.concat([left,right],ignore_index=True),dflist)
    df_res['一月'] = df_res['一月']/10000
    df_res['二月'] = df_res['二月']/10000
    df_res['三月'] = df_res['三月']/10000
    df_res['四月'] = df_res['四月']/10000
    df_res['五月'] = df_res['五月']/10000
    df_res['六月'] = df_res['六月']/10000
    df_res['总销售额'] = df_res['一月'] +df_res['二月'] +df_res['三月'] +df_res['四月']+df_res['五月']+df_res['六月']
    return  df_res.round(0)

# df_yt_2021 = get_xs_by_yt(df2021_ppk)
df_yt_2022 = get_xs_by_yt(df2022_ppk)
df_rj_2021 = get_xs_rijun(df2021_ppk)
df_rj_2022 = get_xs_rijun(df2022_ppk)
dt11,dt12,dt21,dt22=date(2021,1,1),date(2021,7,1),date(2022,1,1),date(2022,7,1)
df_all = get_diff_xs(df2021_ppk,df2022_ppk,0,'南京弘阳广场',dt11,dt12,dt21,dt22)
# df_zcl = get_diff_xs(df2021_ppk,df2022_ppk,1,'南京弘阳广场')
# df_yb = get_diff_xs(df2021_ppk,df2022_ppk,2,'南京弘阳广场')
# df_zcl
df_rj_2021.round(0)


In [ ]:
df_rj_2022

In [ ]:
df_all


In [ ]:
df_yt_2022

In [ ]:
df2022_ppk

### 客流：平日/周末客流统计

In [ ]:
import pandas as pd
import numpy as np

df_1 = pd.read_excel('~/Desktop/销售客流pandas/南京弘阳广场客流202101-06.xls',header=5)
df_a1= df_1.loc[(df_1.属性 =='平时') | (df_1.属性 =='周末')][['日期','星期','属性','P00011']].sort_values(by='日期').rename(columns={'P00011':'客流'})

df_ps1= df_a1.loc[(df_a1.属性 =='平时')][['日期','星期','属性','客流']].sort_values(by='日期')
df_zm1= df_a1.loc[(df_a1.属性 =='周末')][['日期','星期','属性','客流']].sort_values(by='日期')
df_a1['日期'] = pd.to_datetime(df_a1['日期'],format='%Y-%m-%d')
df_a1['月份'] =df_a1['日期'].dt.month
r1 = df_a1.groupby(by=['月份']).agg({'客流':'sum'}).reset_index()

df_ps1=df_ps1.groupby(df_ps1.属性).agg({'客流':'sum'}).reset_index()
df_zm1=df_zm1.groupby(df_zm1.属性).agg({'客流':'sum'}).reset_index()

df_2 = pd.read_excel('~/Desktop/销售客流pandas/南京弘阳广场客流202201-06.xls',header=5)
df_a2= df_2.loc[(df_2.属性 =='平时') | (df_2.属性 =='周末')][['日期','星期','属性','P00011']].sort_values(by='日期').rename(columns={'P00011':'客流'}).reset_index()
df_ps2= df_a2.loc[(df_a2.属性 =='平时')][['日期','星期','属性','客流']].sort_values(by='日期')
df_zm2= df_a2.loc[(df_a2.属性 =='周末')][['日期','星期','属性','客流']].sort_values(by='日期')
df_ps2=df_ps2.groupby(df_ps2.属性).agg({'客流':'sum'}).reset_index()
df_zm2=df_zm2.groupby(df_zm2.属性).agg({'客流':'sum'}).reset_index()

df_a2['日期'] = pd.to_datetime(df_a2['日期'],format='%Y-%m-%d')
df_a2['月份'] =df_a2['日期'].dt.month
r2 = df_a2.groupby(by=['月份']).agg({'客流':'sum'}).reset_index()
# r2.T
(df_ps1,df_zm1)

### 根据品牌库、销售导出数据 计算

In [ ]:
import pandas as pd
import numpy as np
from output_style_format import to_excel_auto_column_weight

def get_df(folderpath,shoppingmall,month:str):
    #folderpath,month1,month2,shoppingmall,form
    df_xs_all = pd.read_excel(folderpath+shoppingmall+month+'.xlsx')
    df_ppk = pd.read_excel(folderpath+'/品牌库.xlsx')


    df_xs_all =df_xs_all[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_ppk = df_ppk[['品牌名称','业态']]

    df_xs_all_ppk = pd.merge(df_xs_all,df_ppk,on='品牌名称',how='left')

    df_xs_zcld = df_xs_all_ppk.loc[(df_xs_all_ppk.业态=='主次力店')]
    df_xs_zcld = df_xs_zcld.groupby(df_xs_zcld.营业额日期).agg({'营业额':'sum'}).reset_index()

    df_xs_ybdp = df_xs_all_ppk.loc[(df_xs_all_ppk.业态!='主次力店')]
    df_xs_ybdp = df_xs_ybdp.groupby(df_xs_ybdp.营业额日期).agg({'营业额':'sum'}).reset_index()

    df_xs_zcld['营业额日期']=df_xs_zcld['营业额日期'].apply(lambda x : x.strftime('%Y-%m-%d'))
    df_xs_ybdp['营业额日期']=df_xs_ybdp['营业额日期'].apply(lambda x : x.strftime('%Y-%m-%d'))

    return df_xs_zcld,df_xs_ybdp

df1,df2=get_df('~/Desktop/销售客流pandas/','合肥弘阳广场','202105')
df5,df6=get_df('~/Desktop/销售客流pandas/','合肥弘阳广场','202205')

with pd.ExcelWriter('~/Desktop/销售客流pandas/xs_合肥弘阳广场1.xlsx',engine="openpyxl") as writer:
    to_excel_auto_column_weight(df1,writer,'主次力店202105')
    to_excel_auto_column_weight(df2,writer,'一般店铺202105')
    to_excel_auto_column_weight(df5,writer,'主次力店202205')
    to_excel_auto_column_weight(df6,writer,'一般店铺202205')



In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime


def get_diff_xs(folderpath,month1,month2,shoppingmall,form):
    """
    Parameters: Str: month1,month2,shoppingmall
    get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',0)
    form:0-所有业态；1-主次力店；2-一般商铺
    Return :
    项目名称	 平日销售日均	 周末销售日均	周末/平日比	周末占比同比
    合肥弘阳广场	788503.48	1123773.68  	0.58	-0.39
    """
    #导入数据并整理
    df_xs_zp = pd.read_excel(folderpath+shoppingmall+month1+'.xlsx')
    df_xs_dj = pd.read_excel(folderpath+shoppingmall+month2+'.xlsx')
    df_ppk = pd.read_excel('~/Desktop/销售客流pandas/品牌库.xlsx')
    df_xs_zp =df_xs_zp[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_xs_dj =df_xs_dj[['项目名称','铺位类型','品牌名称','营业额','营业额日期']]
    df_xs_zp['营业额日期']=df_xs_zp.营业额日期.apply(lambda x:x.date())
    df_xs_dj['营业额日期']=df_xs_dj.营业额日期.apply(lambda x:x.date())

    df_ppk = df_ppk[['品牌名称','业态']]
    df_xs_all_ppk1 = pd.merge(df_xs_zp,df_ppk,on='品牌名称',how='left')
    df_xs_all_ppk2 = pd.merge(df_xs_dj,df_ppk,on='品牌名称',how='left')
    #根据业态，区分销售数据的统计范围
    if form==1:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态=='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态=='主次力店')]
    if form==2:
        df_xs_all_ppk1=df_xs_all_ppk1.loc[(df_xs_all_ppk1.业态!='主次力店')]
        df_xs_all_ppk2=df_xs_all_ppk2.loc[(df_xs_all_ppk2.业态!='主次力店')]

    #month1 周末平日销售结果
    df_xs_all_ppk1['周末'] = df_xs_all_ppk1['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    df_tmp1 = df_xs_all_ppk1.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    df_tmp1['日均营业额']=df_tmp1.apply(lambda x:x['营业额']/9 if x['周末']=='是' else x['营业额']/22,axis=1)

    a,b,c=df_tmp1.iat[0,2],df_tmp1.iat[1,2],df_tmp1.iat[1,1]/df_tmp1.iat[0,1]
    df_xs_res1 = pd.DataFrame(data={'项目名称':['合肥弘阳广场'],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df1 = df_xs_res1.reset_index()

    #month2 周末平日销售结果
    df_xs_all_ppk2['周末'] = df_xs_all_ppk2['营业额日期'].apply(lambda x: '是' if x.weekday() >4 else '否')
    df_tmp2 = df_xs_all_ppk2.groupby(by='周末').agg({'营业额':'sum'}).reset_index()
    df_tmp2['日均营业额']=df_tmp2.apply(lambda x:x['营业额']/9 if x['周末']=='是' else x['营业额']/22,axis=1)

    a,b,c=df_tmp2.iat[0,2],df_tmp2.iat[1,2],df_tmp2.iat[1,1]/df_tmp2.iat[0,1]
    df_xs_res2 = pd.DataFrame(data={'项目名称':['合肥弘阳广场'],'平日销售日均':[a],'周末销售日均':[b],'周末/平日比':[c]})
    df2 = df_xs_res2.reset_index()

    #输出同比结果
    df2['上期周末占比']=df1.iat[0,4]
    df2['周末占比同比']=df2.iat[0,4]-df1.iat[0,4]

    return df2.round(2)

df_all = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',0)
df_zcld = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',1)
df_ybsp = get_diff_xs('~/Desktop/销售客流pandas/','202105','202205','合肥弘阳广场',2)

with pd.ExcelWriter('~/Downloads/xs_hefei.xlsx') as writer:
    df_all.to_excel(writer,sheet_name='所有业态')
    df_zcld.to_excel(writer,sheet_name='主次力店')
    df_ybsp.to_excel(writer,sheet_name='一般商铺')


In [ ]:
import pandas as pd
import numpy as np

df_tj =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='统计表',header=2)
df_hf_202105 =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='主次力店202105')
df_hf_202205 =pd.read_excel('~/Downloads/xs_hf_2205.xlsx',sheet_name='主次力店202205')

df1=df_tj.iloc[:,1:]
df2=df_tj.iloc[:,:4]

with pd.ExcelWriter('~/Downloads/11111.xlsx') as writer:
    df1.to_excel(writer,sheet_name='s1')
    df2.to_excel(writer,sheet_name='s1',startrow=4)

In [ ]:

dd = df_jy[['客户编号']]
dd

In [ ]:
import pandas as pd

# 设定最大显示行数、列数为10000
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 10000)

In [ ]:
import pandas as pd
import numpy as np

df_jy = pd.read_excel('~/Downloads/客户/客户_竞优20220715.xls')
dd_jy = df_jy[['客户编码','客户姓名','法人身份证号码']]
df_zz = pd.read_excel('~/Downloads/客户/客户_总账20220715.xlsx')
df_jd = pd.read_excel('~/Downloads/客户/客户_金蝶20220715.xlsx')

# dd_jd = df_jd[['客户编号','名称']]
df = pd.merge(df_jd,df_zz,on='客户编码',how='left') # 先筛选金蝶中没被总账引用的客户，然后拿筛选结果去对比竞优的数据，对比结果中竞优没有的就要 反核准。

df1 =  df.loc[(pd.isna(df.创建人))] # df1 金蝶中未被总账使用的客户

df2 = pd.merge(df1,df_jy,on='客户编码',how='left')
df3  = df2.loc[(pd.isna(df2.客户姓名))]
# with pd.ExcelWriter('~/Downloads/客户/竞优无-07151616.xlsx') as writer:
#     df3.to_excel(writer,sheet_name='1')
df3

In [ ]:
import pandas as pd
import numpy as np

df_jczl = pd.read_excel('~/Downloads/客户/jczl.xlsx')
df_jczl = df_jczl[['编码','名称','税务登记号','创建时间']]
df_jczl['客户编码'] = df_jczl['编码']

df_cwzl = pd.read_excel('~/Downloads/客户/cwzl.xlsx')
df_cwzl = df_cwzl[['客户编码','财务组织编码']]
df = pd.merge(df_jczl,df_cwzl,on='客户编码',how='right')

df =  df.loc[(pd.isna(df.名称))]

with pd.ExcelWriter('~/Downloads/客户/竞优无.xlsx') as writer:
    df.to_excel(writer,sheet_name='1')

In [ ]:
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts


table = Table()

headers = ["City name", "Area", "Population", "Annual Rainfall"]
rows = [
    ["Brisbane", 5905, 1857594, 1146.4],
    ["Adelaide", 1295, 1158259, 600.5],
    ["Darwin", 112, 120900, 1714.7],
    ["Hobart", 1357, 205556, 619.5],
    ["Sydney", 2058, 4336374, 1214.8],
    ["Melbourne", 1566, 3806092, 646.9],
    ["Perth", 5386, 1554769, 869.4],
]
table.add(headers, rows)
table.set_global_opts(
    title_opts=ComponentTitleOpts(title="Table-基本示例", subtitle="我是副标题支持换行哦")
)
table.render_notebook()

In [20]:
from pyecharts import options as opts
from pyecharts.charts import Sankey

nodes = [
    {"name": "category1"},
    {"name": "category2"},
    {"name": "category3"},
    {"name": "category4"},
    {"name": "category5"},
    {"name": "category6"},
]

links = [
    {"source": "category1", "target": "category2", "value": 10},
    {"source": "category2", "target": "category3", "value": 15},
    {"source": "category3", "target": "category4", "value": 20},
    {"source": "category5", "target": "category6", "value": 25},
]
c = (
    Sankey()
    .add(
        "sankey",
        nodes,
        links,
        linestyle_opt=opts.LineStyleOpts(opacity=0.2, curve=0.5, color="source"),
        label_opts=opts.LabelOpts(position="right"),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="Sankey-基本示例"))
    .render_notebook()
)

c

In [ ]:
df = pd.read_clipboard()
df


In [ ]:

(
    df.loc[:, ['content']] # 选取标识列，与下行时间转换后形成新的 df
    .assign(time=pd.to_datetime(df['date'] + ' ' +df['time']))
    # 时间分组，每两分钟分组到一起
    .groupby(pd.Grouper(freq='2Min', key='time'))
    # 聚合方法为计算标识的数量
    .apply(lambda x: x.content.value_counts())
    .unstack() # 解除堆叠
    .fillna(0)
    .astype(int).reset_index()
)

In [ ]:
index = pd.MultiIndex.from_tuples([('one', 'a'), ('one', 'b'),
                                   ('two', 'a'), ('two', 'b')])
s = pd.Series(np.arange(1.0, 5.0), index=index)

s

In [ ]:
f=s.T
f

In [ ]:
s.unstack(0)

In [ ]:
pd.date_range("2022-07-06", periods=10, freq="B")

In [ ]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})

table = pd.pivot_table(df, values='D', index=['A', 'B'],
                    columns=['C'], aggfunc=np.sum)

df

In [ ]:
import pandas as pd

df = pd.read_clipboard()
df

In [ ]:
user_info_reset_index = df
user_info_reset_index.melt(id_vars=['name'], value_vars=['sex'])

In [ ]:
import pandas as pd
import numpy as np

df_wu = pd.read_excel('~/Downloads/客户/金蝶有竞优无-金蝶总账未引用.xlsx')

df_jd = pd.read_excel('~/Downloads/客户/客户20220715172039.xlsx')

# dd_jd = df_jd[['客户编号','名称']]
df = pd.merge(df_wu,df_jd,on='客户编码',how='left') # 先筛选金蝶中没被总账引用的客户，然后拿筛选结果去对比竞优的数据，对比结果中竞优没有的就要 反核准。

df1 =  df.loc[(pd.isna(df.名称2))] # df1 金蝶中未被总账使用的客户

df1


In [ ]:
import pandas as pd
import tushare as ts
import numpy as np
pro =  ts.pro_api('5e4e8f2545d9be88ec337aa1f798038a7d0cf9751eafc5afa13e6a08')
df = pro.daily(ts_code='300274.sz', start_date='20180701', end_date='20180718')
df

In [ ]:
import baostock as bs
import pandas as pd

#### 登陆系统 ####
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

#### 获取沪深A股历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
rs = bs.query_history_k_data_plus("300274.sz",
    "date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST",
    start_date='2017-07-01', end_date='2018-7-18',
    frequency="d", adjustflag="3")
print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)
#### 打印结果集 ####
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)

#### 结果集输出到csv文件 ####   
result.to_csv("~/Downloads/history_A_stock_k_data.csv", index=False)
print(result)

#### 登出系统 ####
bs.logout()


In [ ]:
import baostock as bs
import pandas as pd

#### 登陆系统 ####
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

#### 获取沪深A股历史K线数据 ####
# 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。“分钟线”不包含指数。
# 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
# 周月线指标：date,code,open,high,low,close,volume,amount,adjustflag,turn,pctChg
rs = bs.query_history_k_data_plus("sz.300274",
    "date,time,code,open,high,low,close,volume,amount,adjustflag",
    start_date='2022-07-01', end_date='2022-07-16',
    frequency="5", adjustflag="3")
print('query_history_k_data_plus respond error_code:'+rs.error_code)
print('query_history_k_data_plus respond  error_msg:'+rs.error_msg)

#### 打印结果集 ####
data_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    data_list.append(rs.get_row_data())
result = pd.DataFrame(data_list, columns=rs.fields)

#### 结果集输出到csv文件 ####   
result.to_excel("~/Downloads/history_A_stock_k_data.xlsx", index=False)
print(result)

#### 登出系统 ####
bs.logout()

In [ ]:
import baostock as bs
import pandas as pd

# 登陆系统
lg = bs.login()
# 显示登陆返回信息
print('login respond error_code:'+lg.error_code)
print('login respond  error_msg:'+lg.error_msg)

# 获取行业分类数据
rs = bs.query_stock_industry()
# rs = bs.query_stock_basic(code_name="浦发银行")
print('query_stock_industry error_code:'+rs.error_code)
print('query_stock_industry respond  error_msg:'+rs.error_msg)

# 打印结果集
industry_list = []
while (rs.error_code == '0') & rs.next():
    # 获取一条记录，将记录合并在一起
    industry_list.append(rs.get_row_data())
result = pd.DataFrame(industry_list, columns=rs.fields)
# 结果集输出到csv文件
result.to_excel("~/Downloads/hyfl.xlsx", encoding="gbk", index=False)
print(result)

# 登出系统
bs.logout()


In [ ]:
bs.login()
date = '2022-07-15'
# 获取指定日期的指数、股票数据
stock_rs = bs.query_all_stock('2022-07-15')
stock_df = stock_rs.get_data()
data_df = pd.DataFrame()
for code in stock_df["code"]:
    print("Downloading :" + code)
    k_rs = bs.query_history_k_data_plus(code, "date,code,open,high,low,close", date, date)
    data_df = data_df.append(k_rs.get_data())
bs.logout()
data_df.to_excel("~/Downloads/allstockbydate.xlsx", encoding="gbk", index=False)
print(data_df)


In [1]:
from jqdatasdk import *

auth('17712910405','Momocha_1982') #账号是申请时所填写的手机号；密码为聚宽官网登录密码


auth success 


In [ ]:
# df = get_bars('002796.XSHE', 5, unit='1d',fields=['date','open','high','low','close'],include_now=False,end_dt='2022-8-05')
df = get_price('002796.XSHE', end_date='2022-07-19 09:32:00',count=4, frequency='daily', fields=['open','close','high','low','volume','money','high_limit','paused'])
df

In [ ]:
get_security_info('000001.XSHE')

In [ ]:
normalize_code('000625')

In [ ]:
security ='000625.XSHE'
# 获取股票的收盘价
close_data5 = get_bars(security, count=5, unit='1d', fields=['close'])
close_data10 = get_bars(security, count=10, unit='1d', fields=['close'])
# 取得过去五天的平均价格
MA5 = close_data5['close'].mean()
MA10 = close_data10['close'].mean()
MA5,MA10

In [8]:
import pandas as pd
ds = get_concept_stocks('SC0053', date=None)

display_name = []
code = []
for ss in ds:
    display_name.append(get_security_info(ss).display_name)
    code.append(ss)
ddf = pd.DataFrame(display_name,code)
ddf.to_excel('~/Downloads/SC0053.xlsx')

In [13]:
ddff = get_price(ds, start_date='2022-08-02', end_date='2022-08-02', frequency='daily', fields=['open','close','high','high_limit','low','volume','money'], skip_paused=False, fq='pre', count=None)

/Users/zif/opt/anaconda3/lib/python3.9/site-packages/jqdatasdk/api.py:34: PanelObsoleteWarning: 当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口的 panel 参数将固定为 False（0.25 及以上版本的 pandas 不再支持 panel，如使用该数据结构和相关函数请注意修改）
  warnings.warn(PandasChecker.VERSION_NOTICE_MESSAGE, PanelObsoleteWarning)


In [14]:
ddff.to_excel('~/Downloads/SC0053ss.xlsx')

In [ ]:
import pandas as pd
lst = get_concept_stocks('SC0232', date=None)
dff =  pd.DataFrame(lst)
# dff.to_excel('~/Downloads/20220718333.xlsx')
[get_stock_info(s) for s in lst]

In [ ]:
get_concept('002796.XSHE', date='2022-07-15')

get_industry('002796.XSHE', date='2022-07-15')

In [ ]:
dff =get_call_auction('002796.XSHE','2022-07-02','2022-07-22')
dff

In [ ]:
df=get_mtss('002796.XSHE','2022-07-02','2022-07-22',fields=['date','sec_code','fin_value','fin_buy_value','fin_refund_value','sec_value','sec_sell_value','sec_refund_value','fin_sec_value'])
print(df[:5])

In [ ]:
d = get_ticks("000001.XSHE",start_dt=None, end_dt="2018-07-03", count=8)
print(d)

In [ ]:
import datetime as dt

date = list(get_trade_days(start_date='2012-1-1', end_date=dt.date.today()))
stock_list = ['002796.XSHE']

In [ ]:
for stock in stock_list:
    df = get_price(stock, start_date='2012-1-1', end_date=dt.date.today(), frequency='1d', fields=None, 
                   skip_paused=True, fq='pre', panel=False, )
    df['week'] = df.index.week
    df['year'] = df.index.year
    week_stock = df.groupby(['year', 'week'])[['open']].first()
    week_stock['close'] = df.groupby(['year', 'week'])['close'].last()
    week_stock['high'] = df.groupby(['year', 'week'])['high'].max()
    week_stock['low'] = df.groupby(['year', 'week'])['low'].min()
    week_stock['EMA_short'] = week_stock['close'].ewm(span=12, adjust=False).mean()
    week_stock['EMA_long'] = week_stock['close'].ewm(span=26, adjust=False).mean()
    week_stock['DIF'] = week_stock['EMA_short'] - week_stock['EMA_long']
    week_stock['DEA'] = week_stock['DIF'].ewm(span=9, adjust=False).mean()
    week_stock['MACD'] = 2*(week_stock['DIF'] - week_stock['DEA'])
    week_stock.to_excel('~/Downloads/macd20220718.xlsx')


In [2]:
def _get_file_content_ths(file: str = "ths.js") -> str:
    """
    获取 JS 文件的内容
    :param file:  JS 文件名
    :type file: str
    :return: 文件内容
    :rtype: str
    """
    setting_file_path = get_ths_js(file)
    with open(setting_file_path) as f:
        file_data = f.read()
    return file_data

In [6]:
import akshare as ak
ak.stock_board_cons_ths('301558')

# https://tv.cctv.com/lm/xwlb/

,序号,代码,名称,现价,涨跌幅,涨跌,涨速,换手,量比,振幅,成交额,流通股,流通市值,市盈率
0,1,300785,值得买,31.35,4.57,1.37,0.32,20.08,3.32,14.61,4.71亿,0.75亿,23.60亿,103.24
1,2,002027,分众传媒,6.06,3.59,0.21,0.17,1.4,3.06,4.96,12.21亿,144.42亿,875.20亿,31.18
2,3,301110,青木股份,39.97,3.42,1.32,0.0,11.7,3.08,6.18,0.70亿,0.15亿,6.07亿,41.38
3,4,002044,美年健康,4.83,2.55,0.12,0.21,1.3,2.7,4.25,2.45亿,38.71亿,186.98亿,--
4,5,002153,石基信息,13.03,2.36,0.3,-0.08,0.56,1.7,4.56,0.88亿,12.30亿,160.31亿,395.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,99,002139,拓邦股份,12.47,-8.78,-1.2,0.0,6.49,1.54,10.24,8.63亿,10.42亿,129.90亿,32.15
99,100,000034,神州数码,17.58,-9.98,-1.95,0.0,5.06,1.96,9.37,4.96亿,5.42亿,95.31亿,14.94
100,101,002335,科华数据,34.77,-9.99,-3.86,0.0,6.81,1.06,11.49,9.74亿,3.98亿,138.28亿,48.31
101,102,300274,阳光电源,111.4,-12.48,-15.88,-0.26,5.57,2.05,13.74,72.96亿,11.28亿,1256.85亿,91.87


In [3]:
import pandas as pd
from datetime import datetime
from functools import lru_cache

import requests
from bs4 import BeautifulSoup
from py_mini_racer import py_mini_racer
from tqdm import tqdm

from akshare.utils import demjson
from akshare.datasets import get_ths_js

def stock_board_cons_ths_modified(symbol: str = "301558") -> pd.DataFrame:
    """
    通过输入行业板块或者概念板块的代码获取成份股
    http://q.10jqka.com.cn/thshy/detail/code/881121/
    http://q.10jqka.com.cn/gn/detail/code/301558/
    :param symbol: 行业板块或者概念板块的代码
    :type symbol: str
    :return: 行业板块或者概念板块的成份股
    :rtype: pandas.DataFrame
    """
    print('就问你服不服')
    js_code = py_mini_racer.MiniRacer()
    js_content = _get_file_content_ths("ths.js")
    js_code.eval(js_content)
    v_code = js_code.call("v")
    headers = {
        # "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
        "Cookie": f"v={v_code}",
    }
    url = f"http://q.10jqka.com.cn/thshy/detail/field/199112/order/desc/page/1/ajax/1/code/{symbol}"
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "lxml")
    url_flag = "thshy"
    if soup.find("td", attrs={"colspan": "14"}):
        url = f"http://q.10jqka.com.cn/gn/detail/field/199112/order/desc/page/1/ajax/1/code/{symbol}"
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.text, "lxml")
        url_flag = "gn"
    try:
        page_num = int(
            soup.find_all("a", attrs={"class": "changePage"})[-1]["page"]
        )
    except IndexError as e:
        page_num = 1
    big_df = pd.DataFrame()
    for page in tqdm(range(1, page_num + 1), leave=False):
        v_code = js_code.call("v")
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36",
            "Cookie": f"v={v_code}",
        }
        url = f"http://q.10jqka.com.cn/{url_flag}/detail/field/199112/order/desc/page/{page}/ajax/1/code/{symbol}"
        r = requests.get(url, headers=headers)
        temp_df = pd.read_html(r.text)[0]
        big_df = pd.concat([big_df, temp_df], ignore_index=True)
    big_df.rename(
        {
            "涨跌幅(%)": "涨跌幅",
            "涨速(%)": "涨速",
            "换手(%)": "换手",
            "振幅(%)": "振幅",
        },
        inplace=True,
        axis=1,
    )
    del big_df["加自选"]
    big_df["代码"] = big_df["代码"].astype(str).str.zfill(6)
    return big_df